# Song Database Creation

## Overview

### Deliverable
- Audio features dataframe with minimum of 1000 songs. Although the more song the better. A few thousand would be good.

### Notes
- for getting audio features, each user account has a limited number of requests. Sending too many requests will result in you getting locked out. To avoid this make sure to divide your track list in chunks and send one request for your whole playlist

### Ideas
- Create function to make the process repeatable for other playlist

### Steps
1. get playlist_id
   - playlist_id can be copied from url
   - or use sp.search(q=“r&b, limit=3,type=“playlist”)
2. get playlist tracks
3. get track ids
4. chop id's into chunks
5. get audio features of each chunk
6. create dataframe
7. csv export

#### Importing functions credentials and data

In [12]:
#Libraries and functions
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import pprint

#importing spotify API credentials
import config

In [13]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

#### 1. Get playlist id

In [32]:
everything_good_playlist_id = '1v94bqukAqC4IqLTERiljX'
everything_good_user_name = "1167792454"

#pprint.pprint(everything_good)

#### 2. Get playlist tracks

In [11]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [2]:
tracks = get_playlist_tracks(everything_good_user_name,everything_good_playlist_id)

NameError: name 'everything_good_user_name' is not defined

#### 3. Get track ids

In [97]:
#testing for 1 track
tracks["tracks"]["items"][0]["track"]["id"]

TypeError: list indices must be integers or slices, not str

In [98]:
#for all
track_ids = []
for i in tracks:
    track_ids.append(i['track']['id'])
#track_ids

#### 4. Chopping track ids into chunks

In [99]:
chunk_size = 100
track_ids_chopped = [track_ids[i:i+chunk_size] for i in range(0,len(track_ids), 100)]
#track_ids_chopped

#### 5. Get audio features

In [100]:
af_list = []
for i in track_ids_chopped:
    af_list.extend(sp.audio_features(i))


In [101]:
af_list

[{'danceability': 0.382,
  'energy': 0.451,
  'key': 7,
  'loudness': -6.606,
  'mode': 0,
  'speechiness': 0.0293,
  'acousticness': 0.557,
  'instrumentalness': 3.39e-05,
  'liveness': 0.265,
  'valence': 0.506,
  'tempo': 150.803,
  'type': 'audio_features',
  'id': '6yzJiHzsscf0n88xb9aNT4',
  'uri': 'spotify:track:6yzJiHzsscf0n88xb9aNT4',
  'track_href': 'https://api.spotify.com/v1/tracks/6yzJiHzsscf0n88xb9aNT4',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6yzJiHzsscf0n88xb9aNT4',
  'duration_ms': 267540,
  'time_signature': 4},
 {'danceability': 0.742,
  'energy': 0.375,
  'key': 7,
  'loudness': -13.829,
  'mode': 1,
  'speechiness': 0.0482,
  'acousticness': 0.397,
  'instrumentalness': 0.901,
  'liveness': 0.0996,
  'valence': 0.105,
  'tempo': 102.023,
  'type': 'audio_features',
  'id': '5zdXRoKLZ54VBdbdn7yhz4',
  'uri': 'spotify:track:5zdXRoKLZ54VBdbdn7yhz4',
  'track_href': 'https://api.spotify.com/v1/tracks/5zdXRoKLZ54VBdbdn7yhz4',
  'analysis_url': 'https

#### 6. Create dataframe

In [102]:
df = pd.DataFrame(af_list)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.382,0.451,7,-6.606,0,0.0293,0.5570,0.000034,0.2650,0.506,150.803,audio_features,6yzJiHzsscf0n88xb9aNT4,spotify:track:6yzJiHzsscf0n88xb9aNT4,https://api.spotify.com/v1/tracks/6yzJiHzsscf0...,https://api.spotify.com/v1/audio-analysis/6yzJ...,267540,4
1,0.742,0.375,7,-13.829,1,0.0482,0.3970,0.901000,0.0996,0.105,102.023,audio_features,5zdXRoKLZ54VBdbdn7yhz4,spotify:track:5zdXRoKLZ54VBdbdn7yhz4,https://api.spotify.com/v1/tracks/5zdXRoKLZ54V...,https://api.spotify.com/v1/audio-analysis/5zdX...,342163,4
2,0.585,0.552,0,-10.613,1,0.0353,0.1940,0.017100,0.0735,0.496,100.089,audio_features,5qvCKAyjgHnpR5c63OedCY,spotify:track:5qvCKAyjgHnpR5c63OedCY,https://api.spotify.com/v1/tracks/5qvCKAyjgHnp...,https://api.spotify.com/v1/audio-analysis/5qvC...,267813,4
3,0.405,0.535,6,-7.504,0,0.0531,0.0531,0.106000,0.0696,0.637,97.698,audio_features,2vByA9JgfMU2bPjUqmxCdv,spotify:track:2vByA9JgfMU2bPjUqmxCdv,https://api.spotify.com/v1/tracks/2vByA9JgfMU2...,https://api.spotify.com/v1/audio-analysis/2vBy...,210800,4
4,0.586,0.526,0,-6.928,1,0.0564,0.5550,0.000000,0.2340,0.456,107.153,audio_features,0pPQwIjwqduvLfZORYYEaI,spotify:track:0pPQwIjwqduvLfZORYYEaI,https://api.spotify.com/v1/tracks/0pPQwIjwqduv...,https://api.spotify.com/v1/audio-analysis/0pPQ...,161267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,0.744,0.831,7,-6.004,0,0.0300,0.0250,0.000236,0.0728,0.702,129.968,audio_features,0aWhxzFrP2Yyz3CetJe0xE,spotify:track:0aWhxzFrP2Yyz3CetJe0xE,https://api.spotify.com/v1/tracks/0aWhxzFrP2Yy...,https://api.spotify.com/v1/audio-analysis/0aWh...,228066,4
315,0.609,0.797,1,-5.945,0,0.0361,0.0244,0.345000,0.4430,0.546,120.039,audio_features,5yzqu20ssqs9ric3fHO74I,spotify:track:5yzqu20ssqs9ric3fHO74I,https://api.spotify.com/v1/tracks/5yzqu20ssqs9...,https://api.spotify.com/v1/audio-analysis/5yzq...,216674,4
316,0.594,0.608,1,-8.930,1,0.0410,0.0259,0.122000,0.1560,0.639,169.969,audio_features,2j5AyPkjugfvw0HWrK1Wvr,spotify:track:2j5AyPkjugfvw0HWrK1Wvr,https://api.spotify.com/v1/tracks/2j5AyPkjugfv...,https://api.spotify.com/v1/audio-analysis/2j5A...,141503,4
317,0.811,0.452,7,-8.676,1,0.1150,0.1620,0.003050,0.1200,0.556,121.058,audio_features,0BWBqb5XxraObLopqskk6D,spotify:track:0BWBqb5XxraObLopqskk6D,https://api.spotify.com/v1/tracks/0BWBqb5XxraO...,https://api.spotify.com/v1/audio-analysis/0BWB...,199006,4


#### 7. Csv export

In [ ]:
df.to_csv("../data/everything_good_af.csv")

## Creating function to make process repeatable

### Defining function

In [14]:
def playlist_audio_features(username, playlist_id):
    # Get tracks
    tracks = get_playlist_tracks(username, playlist_id)
    
    # Get track ids, skipping None
    track_ids = [i['track']['id'] for i in tracks if i['track']['id'] is not None]
        
    # Chopping track ids list in chunks of 100 songs 
    chunk_size = 100
    track_ids_chopped = [track_ids[i:i + chunk_size] for i in range(0, len(track_ids), chunk_size)]
    
    # Get audio features
    af_list = []
    for chunk in track_ids_chopped:
        try:
            af_list.extend(sp.audio_features(chunk))
        except TypeError as e:
            print(f"Error retrieving audio features for chunk: {e}")
    
    # Create dataframe
    af_df = pd.DataFrame(af_list)

    return af_df

### Testing the function with previous playlist

In [33]:
everything_good_af_df = playlist_audio_features(everything_good_user_name,everything_good_playlist_id)

In [34]:
everything_good_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.382,0.451,7,-6.606,0,0.0293,0.5570,0.000034,0.2650,0.506,150.803,audio_features,6yzJiHzsscf0n88xb9aNT4,spotify:track:6yzJiHzsscf0n88xb9aNT4,https://api.spotify.com/v1/tracks/6yzJiHzsscf0...,https://api.spotify.com/v1/audio-analysis/6yzJ...,267540,4
1,0.742,0.375,7,-13.829,1,0.0482,0.3970,0.901000,0.0996,0.105,102.023,audio_features,5zdXRoKLZ54VBdbdn7yhz4,spotify:track:5zdXRoKLZ54VBdbdn7yhz4,https://api.spotify.com/v1/tracks/5zdXRoKLZ54V...,https://api.spotify.com/v1/audio-analysis/5zdX...,342163,4
2,0.585,0.552,0,-10.613,1,0.0353,0.1940,0.017100,0.0735,0.496,100.089,audio_features,5qvCKAyjgHnpR5c63OedCY,spotify:track:5qvCKAyjgHnpR5c63OedCY,https://api.spotify.com/v1/tracks/5qvCKAyjgHnp...,https://api.spotify.com/v1/audio-analysis/5qvC...,267813,4
3,0.405,0.535,6,-7.504,0,0.0531,0.0531,0.106000,0.0696,0.637,97.698,audio_features,2vByA9JgfMU2bPjUqmxCdv,spotify:track:2vByA9JgfMU2bPjUqmxCdv,https://api.spotify.com/v1/tracks/2vByA9JgfMU2...,https://api.spotify.com/v1/audio-analysis/2vBy...,210800,4
4,0.586,0.526,0,-6.928,1,0.0564,0.5550,0.000000,0.2340,0.456,107.153,audio_features,0pPQwIjwqduvLfZORYYEaI,spotify:track:0pPQwIjwqduvLfZORYYEaI,https://api.spotify.com/v1/tracks/0pPQwIjwqduv...,https://api.spotify.com/v1/audio-analysis/0pPQ...,161267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,0.744,0.831,7,-6.004,0,0.0300,0.0250,0.000236,0.0728,0.702,129.968,audio_features,0aWhxzFrP2Yyz3CetJe0xE,spotify:track:0aWhxzFrP2Yyz3CetJe0xE,https://api.spotify.com/v1/tracks/0aWhxzFrP2Yy...,https://api.spotify.com/v1/audio-analysis/0aWh...,228066,4
315,0.609,0.797,1,-5.945,0,0.0361,0.0244,0.345000,0.4430,0.546,120.039,audio_features,5yzqu20ssqs9ric3fHO74I,spotify:track:5yzqu20ssqs9ric3fHO74I,https://api.spotify.com/v1/tracks/5yzqu20ssqs9...,https://api.spotify.com/v1/audio-analysis/5yzq...,216674,4
316,0.594,0.608,1,-8.930,1,0.0410,0.0259,0.122000,0.1560,0.639,169.969,audio_features,2j5AyPkjugfvw0HWrK1Wvr,spotify:track:2j5AyPkjugfvw0HWrK1Wvr,https://api.spotify.com/v1/tracks/2j5AyPkjugfv...,https://api.spotify.com/v1/audio-analysis/2j5A...,141503,4
317,0.811,0.452,7,-8.676,1,0.1150,0.1620,0.003050,0.1200,0.556,121.058,audio_features,0BWBqb5XxraObLopqskk6D,spotify:track:0BWBqb5XxraObLopqskk6D,https://api.spotify.com/v1/tracks/0BWBqb5XxraO...,https://api.spotify.com/v1/audio-analysis/0BWB...,199006,4


In [35]:
everything_good_af_df.to_csv("../data/everything_good_af.csv")

## Adding more playlists

### Greatestest hip hop songs of all time

In [36]:
greatest_hiphop_playlist_id = "29S1jCDVUgUlF4baQA7wxB"
greatest_hiphop_username = "KD"
greatest_hiphop_af_df = playlist_audio_features(greatest_hiphop_username,greatest_hiphop_playlist_id)
greatest_hiphop_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.532,0.819,6,-5.282,0,0.2560,0.01570,0.000000,0.3710,0.594,90.215,audio_features,1fotoYONO343JjbC8XvPSl,spotify:track:1fotoYONO343JjbC8XvPSl,https://api.spotify.com/v1/tracks/1fotoYONO343...,https://api.spotify.com/v1/audio-analysis/1fot...,247333,4
1,0.716,0.531,7,-7.355,1,0.1220,0.07030,0.000000,0.2240,0.344,71.994,audio_features,74tLlkN3rgVzRqQJgPfink,spotify:track:74tLlkN3rgVzRqQJgPfink,https://api.spotify.com/v1/tracks/74tLlkN3rgVz...,https://api.spotify.com/v1/audio-analysis/74tL...,386907,4
2,0.746,0.978,9,-5.324,1,0.0978,0.05550,0.000041,0.0260,0.653,153.809,audio_features,3WibbMr6canxRJXhNtAvLU,spotify:track:3WibbMr6canxRJXhNtAvLU,https://api.spotify.com/v1/tracks/3WibbMr6canx...,https://api.spotify.com/v1/audio-analysis/3Wib...,304227,4
3,0.596,0.620,5,-6.133,0,0.0390,0.01220,0.000000,0.8200,0.102,80.029,audio_features,0mEdbdeRFQwBhN4xfyIeUM,spotify:track:0mEdbdeRFQwBhN4xfyIeUM,https://api.spotify.com/v1/tracks/0mEdbdeRFQwB...,https://api.spotify.com/v1/audio-analysis/0mEd...,271600,4
4,0.775,0.734,7,-5.410,1,0.2150,0.00138,0.000000,0.3040,0.342,90.001,audio_features,2LhhnqrHPWvXYveZVHLKrd,spotify:track:2LhhnqrHPWvXYveZVHLKrd,https://api.spotify.com/v1/tracks/2LhhnqrHPWvX...,https://api.spotify.com/v1/audio-analysis/2Lhh...,210840,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0.935,0.780,6,-5.795,0,0.1970,0.01580,0.000000,0.1640,0.918,123.895,audio_features,1wLhwzPAtze5JT8qqXlqaO,spotify:track:1wLhwzPAtze5JT8qqXlqaO,https://api.spotify.com/v1/tracks/1wLhwzPAtze5...,https://api.spotify.com/v1/audio-analysis/1wLh...,226160,4
231,0.886,0.620,5,-5.854,1,0.3070,0.08200,0.000000,0.1000,0.600,76.035,audio_features,7cVmKBwzPsh4Fmb6SplfEm,spotify:track:7cVmKBwzPsh4Fmb6SplfEm,https://api.spotify.com/v1/tracks/7cVmKBwzPsh4...,https://api.spotify.com/v1/audio-analysis/7cVm...,229813,4
232,0.849,0.755,1,-7.400,0,0.3630,0.34600,0.000000,0.0564,0.393,144.188,audio_features,7EcE5yCPVZaZut1JqowbcI,spotify:track:7EcE5yCPVZaZut1JqowbcI,https://api.spotify.com/v1/tracks/7EcE5yCPVZaZ...,https://api.spotify.com/v1/audio-analysis/7EcE...,206655,4
233,0.821,0.856,6,-3.172,0,0.2660,0.04750,0.000000,0.3470,0.533,91.023,audio_features,6SSJcmQgBEvHqqCivyVfqG,spotify:track:6SSJcmQgBEvHqqCivyVfqG,https://api.spotify.com/v1/tracks/6SSJcmQgBEvH...,https://api.spotify.com/v1/audio-analysis/6SSJ...,225373,4


In [37]:
#export to csv
greatest_hiphop_af_df.to_csv("../data/greatest_hiphop_songs_of_all_time_af.csv")

### The best of 80s

In [38]:
best_of_80s_playlist_id = "49l9GbetRtsnQC7z3kkiZw"
best_of_80s_username = "Keegan - Old Time Playlists"
best_of_80s_af_df = playlist_audio_features(best_of_80s_username,best_of_80s_playlist_id)
best_of_80s_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.710,0.799,6,-4.897,1,0.0328,0.2200,0.000602,0.3490,0.725,120.372,audio_features,4y1LsJpmMti1PfRQV9AWWe,spotify:track:4y1LsJpmMti1PfRQV9AWWe,https://api.spotify.com/v1/tracks/4y1LsJpmMti1...,https://api.spotify.com/v1/audio-analysis/4y1L...,238267,4
1,0.573,0.902,6,-7.638,0,0.0540,0.0180,0.001250,0.0928,0.876,84.412,audio_features,2WfaOiMkCvy7F5fcp2zZ8L,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L,https://api.spotify.com/v1/tracks/2WfaOiMkCvy7...,https://api.spotify.com/v1/audio-analysis/2Wfa...,225280,4
2,0.645,0.795,7,-12.095,1,0.0527,0.3470,0.003890,0.1040,0.535,112.067,audio_features,4RvWPyQ5RL0ao9LPZeSouE,spotify:track:4RvWPyQ5RL0ao9LPZeSouE,https://api.spotify.com/v1/tracks/4RvWPyQ5RL0a...,https://api.spotify.com/v1/audio-analysis/4RvW...,251489,4
3,0.650,0.645,2,-7.823,0,0.0500,0.0249,0.000016,0.1540,0.328,120.034,audio_features,3z9W9fT8TFFeBY3t5WGLmm,spotify:track:3z9W9fT8TFFeBY3t5WGLmm,https://api.spotify.com/v1/tracks/3z9W9fT8TFFe...,https://api.spotify.com/v1/audio-analysis/3z9W...,186000,4
4,0.671,0.373,9,-18.064,1,0.0323,0.2570,0.000080,0.0481,0.732,92.718,audio_features,2374M0fQpWi3dLnB54qaLX,spotify:track:2374M0fQpWi3dLnB54qaLX,https://api.spotify.com/v1/tracks/2374M0fQpWi3...,https://api.spotify.com/v1/audio-analysis/2374...,295893,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,0.658,0.866,6,-6.337,0,0.0352,0.0504,0.020100,0.0394,0.848,104.898,audio_features,0GZzK43dJ6uNjOuitlCxDr,spotify:track:0GZzK43dJ6uNjOuitlCxDr,https://api.spotify.com/v1/tracks/0GZzK43dJ6uN...,https://api.spotify.com/v1/audio-analysis/0GZz...,227680,4
383,0.763,0.352,0,-11.789,1,0.0555,0.2610,0.000084,0.0791,0.732,83.993,audio_features,3vYrrMpsebx3OJ1H5E1lWQ,spotify:track:3vYrrMpsebx3OJ1H5E1lWQ,https://api.spotify.com/v1/tracks/3vYrrMpsebx3...,https://api.spotify.com/v1/audio-analysis/3vYr...,218427,4
384,0.641,0.619,11,-10.427,0,0.0418,0.1890,0.160000,0.3490,0.434,159.925,audio_features,6X1mehC9bJ1ExEip4HB79W,spotify:track:6X1mehC9bJ1ExEip4HB79W,https://api.spotify.com/v1/tracks/6X1mehC9bJ1E...,https://api.spotify.com/v1/audio-analysis/6X1m...,191800,4
385,0.542,0.810,7,-7.960,1,0.0304,0.0763,0.000004,0.0683,0.920,177.052,audio_features,0ukySDr4uEzZE36s7t7X8E,spotify:track:0ukySDr4uEzZE36s7t7X8E,https://api.spotify.com/v1/tracks/0ukySDr4uEzZ...,https://api.spotify.com/v1/audio-analysis/0uky...,222613,4


In [39]:
#export to csv
best_of_80s_af_df.to_csv("../data/the_best_of_80s_af.csv")

### The best of 90s

In [40]:
best_of_90s_playlist_id = "5Y4aQH7GJvb1pFmqAoIOix"
best_of_90s_username = "Keegan - Old Time Playlists"
best_of_90s_af_df = playlist_audio_features(best_of_90s_username,best_of_90s_playlist_id)
best_of_90s_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.500,0.824,4,-5.846,0,0.0388,0.000160,0.001610,0.0916,0.539,120.125,audio_features,2RsAajgo0g7bMCHxwH3Sk0,spotify:track:2RsAajgo0g7bMCHxwH3Sk0,https://api.spotify.com/v1/tracks/2RsAajgo0g7b...,https://api.spotify.com/v1/audio-analysis/2RsA...,218920,4
1,0.608,0.419,11,-10.016,1,0.0331,0.602000,0.000008,0.1080,0.602,100.220,audio_features,4vDBJeeQCbhP9FaPPMsYkY,spotify:track:4vDBJeeQCbhP9FaPPMsYkY,https://api.spotify.com/v1/tracks/4vDBJeeQCbhP...,https://api.spotify.com/v1/audio-analysis/4vDB...,268333,4
2,0.472,0.652,1,-10.171,0,0.0305,0.013100,0.000011,0.1400,0.401,140.124,audio_features,0wokCRaKD0zPNhMRXAgVsr,spotify:track:0wokCRaKD0zPNhMRXAgVsr,https://api.spotify.com/v1/tracks/0wokCRaKD0zP...,https://api.spotify.com/v1/audio-analysis/0wok...,340200,4
3,0.549,0.946,7,-4.265,0,0.0377,0.000945,0.002070,0.3680,0.477,135.959,audio_features,1Ab3miDy9TBJskMFlkiB5Z,spotify:track:1Ab3miDy9TBJskMFlkiB5Z,https://api.spotify.com/v1/tracks/1Ab3miDy9TBJ...,https://api.spotify.com/v1/audio-analysis/1Ab3...,214360,4
4,0.666,0.855,9,-5.051,0,0.0295,0.179000,0.000001,0.0987,0.803,125.639,audio_features,31AOj9sFz2gM0O3hMARRBx,spotify:track:31AOj9sFz2gM0O3hMARRBx,https://api.spotify.com/v1/tracks/31AOj9sFz2gM...,https://api.spotify.com/v1/audio-analysis/31AO...,268427,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,0.438,0.905,9,-6.161,1,0.0419,0.000009,0.000054,0.3630,0.723,134.163,audio_features,2V4Bc2I962j7acQj1N0PiQ,spotify:track:2V4Bc2I962j7acQj1N0PiQ,https://api.spotify.com/v1/tracks/2V4Bc2I962j7...,https://api.spotify.com/v1/audio-analysis/2V4B...,162667,4
167,0.682,0.765,1,-5.021,0,0.0395,0.026800,0.000034,0.1880,0.567,90.807,audio_features,2hnMS47jN0etwvFPzYk11f,spotify:track:2hnMS47jN0etwvFPzYk11f,https://api.spotify.com/v1/tracks/2hnMS47jN0et...,https://api.spotify.com/v1/audio-analysis/2hnM...,182747,4
168,0.391,0.951,6,-7.419,1,0.0692,0.012600,0.000378,0.0878,0.468,157.660,audio_features,4LJhJ6DQS7NwE7UKtvcM52,spotify:track:4LJhJ6DQS7NwE7UKtvcM52,https://api.spotify.com/v1/tracks/4LJhJ6DQS7Nw...,https://api.spotify.com/v1/audio-analysis/4LJh...,148360,4
169,0.398,0.927,9,-4.212,1,0.0567,0.000062,0.000177,0.0575,0.311,77.106,audio_features,4dVbhS6OiYvFikshyaQaCN,spotify:track:4dVbhS6OiYvFikshyaQaCN,https://api.spotify.com/v1/tracks/4dVbhS6OiYvF...,https://api.spotify.com/v1/audio-analysis/4dVb...,260027,4


In [41]:
#export to csv
best_of_90s_af_df.to_csv("../data/the_best_of_90s_af.csv")

### The best of 70s

In [42]:
best_of_70s_playlist_id = "7e6gKFwEXMF6uDQzmD9YXn"
best_of_70s_username = "Keegan - Old Time Playlists"
best_of_70s_af_df = playlist_audio_features(best_of_70s_username,best_of_70s_playlist_id)
best_of_70s_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.380,0.917,9,-4.486,1,0.0733,0.003150,0.000138,0.0628,0.849,162.521,audio_features,1fl5TvpPkPjx8zo45HY0UQ,spotify:track:1fl5TvpPkPjx8zo45HY0UQ,https://api.spotify.com/v1/tracks/1fl5TvpPkPjx...,https://api.spotify.com/v1/audio-analysis/1fl5...,213093,4
1,0.445,0.384,10,-14.642,1,0.0417,0.635000,0.000077,0.0539,0.216,123.295,audio_features,3ZE3wv8V3w2T2f7nOCjV0N,spotify:track:3ZE3wv8V3w2T2f7nOCjV0N,https://api.spotify.com/v1/tracks/3ZE3wv8V3w2T...,https://api.spotify.com/v1/audio-analysis/3ZE3...,235987,4
2,0.579,0.508,2,-9.484,1,0.0270,0.005740,0.000494,0.0575,0.609,147.125,audio_features,40riOy7x9W7GXjyGp4pjAv,spotify:track:40riOy7x9W7GXjyGp4pjAv,https://api.spotify.com/v1/tracks/40riOy7x9W7G...,https://api.spotify.com/v1/audio-analysis/40ri...,391376,4
3,0.601,0.532,10,-9.119,1,0.0286,0.432000,0.000006,0.0925,0.341,136.571,audio_features,3gdewACMIVMEWVbyb8O9sY,spotify:track:3gdewACMIVMEWVbyb8O9sY,https://api.spotify.com/v1/tracks/3gdewACMIVME...,https://api.spotify.com/v1/audio-analysis/3gde...,281613,4
4,0.733,0.794,5,-10.023,1,0.0307,0.061400,0.036700,0.3300,0.931,148.174,audio_features,37Tmv4NnfQeb0ZgUC4fOJj,spotify:track:37Tmv4NnfQeb0ZgUC4fOJj,https://api.spotify.com/v1/tracks/37Tmv4NnfQeb...,https://api.spotify.com/v1/audio-analysis/37Tm...,348624,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0.197,0.796,2,-7.335,1,0.0645,0.000541,0.417000,0.1870,0.522,175.020,audio_features,5u5qlnyVaewWugJIjzilIc,spotify:track:5u5qlnyVaewWugJIjzilIc,https://api.spotify.com/v1/tracks/5u5qlnyVaewW...,https://api.spotify.com/v1/audio-analysis/5u5q...,261387,4
281,0.670,0.763,7,-6.864,1,0.0341,0.155000,0.000000,0.0799,0.610,114.398,audio_features,4OJFkrRQqol4FsPesF8eu4,spotify:track:4OJFkrRQqol4FsPesF8eu4,https://api.spotify.com/v1/tracks/4OJFkrRQqol4...,https://api.spotify.com/v1/audio-analysis/4OJF...,235667,4
282,0.609,0.624,7,-11.717,1,0.0292,0.039600,0.000168,0.2770,0.691,123.191,audio_features,4RFqi5JQduiSlRZCVlUDGH,spotify:track:4RFqi5JQduiSlRZCVlUDGH,https://api.spotify.com/v1/tracks/4RFqi5JQduiS...,https://api.spotify.com/v1/audio-analysis/4RFq...,184333,4
283,0.423,0.852,8,-9.269,1,0.0327,0.006500,0.899000,0.3310,0.667,114.710,audio_features,7B265QWYkUeqG2ihVEUZ5D,spotify:track:7B265QWYkUeqG2ihVEUZ5D,https://api.spotify.com/v1/tracks/7B265QWYkUeq...,https://api.spotify.com/v1/audio-analysis/7B26...,167413,4


In [43]:
#export to csv
best_of_70s_af_df.to_csv("../data/the_best_of_70s_af.csv")

### The best of 60s

In [44]:
best_of_60s_playlist_id = "3OeNPjOCG6JXQB2ezRRfW1"
best_of_60s_username = "Keegan - Old Time Playlists"
best_of_60s_af_df = playlist_audio_features(best_of_60s_username,best_of_60s_playlist_id)
best_of_60s_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.400,0.836,7,-6.791,1,0.0359,0.37700,0.000000,0.0652,0.907,75.403,audio_features,01K1zFyjJhG2Qk7PIkSAEA,spotify:track:01K1zFyjJhG2Qk7PIkSAEA,https://api.spotify.com/v1/tracks/01K1zFyjJhG2...,https://api.spotify.com/v1/audio-analysis/01K1...,142160,4
1,0.634,0.630,1,-8.277,0,0.0310,0.44700,0.039000,0.1700,0.489,118.628,audio_features,6H3kDe7CGoWYBabAeVWGiD,spotify:track:6H3kDe7CGoWYBabAeVWGiD,https://api.spotify.com/v1/tracks/6H3kDe7CGoWY...,https://api.spotify.com/v1/audio-analysis/6H3k...,270773,4
2,0.438,0.805,8,-6.237,1,0.0624,0.00255,0.000057,0.0842,0.564,113.253,audio_features,2aoo2jlRnM3A0NyLQqMN2f,spotify:track:2aoo2jlRnM3A0NyLQqMN2f,https://api.spotify.com/v1/tracks/2aoo2jlRnM3A...,https://api.spotify.com/v1/audio-analysis/2aoo...,240800,4
3,0.640,0.663,0,-7.516,1,0.0374,0.20100,0.008060,0.1520,0.663,132.770,audio_features,4BP3uh0hFLFRb5cjsgLqDh,spotify:track:4BP3uh0hFLFRb5cjsgLqDh,https://api.spotify.com/v1/tracks/4BP3uh0hFLFR...,https://api.spotify.com/v1/audio-analysis/4BP3...,140773,4
4,0.563,0.915,7,-5.062,1,0.0800,0.24700,0.000085,0.0613,0.710,136.345,audio_features,1mfV2kv9RwFKVI8kEQlmOJ,spotify:track:1mfV2kv9RwFKVI8kEQlmOJ,https://api.spotify.com/v1/tracks/1mfV2kv9RwFK...,https://api.spotify.com/v1/audio-analysis/1mfV...,142227,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0.668,0.253,9,-11.352,1,0.0412,0.59400,0.000042,0.3530,0.672,119.253,audio_features,1vcqtVEolygBkOjxtUV1Kb,spotify:track:1vcqtVEolygBkOjxtUV1Kb,https://api.spotify.com/v1/tracks/1vcqtVEolygB...,https://api.spotify.com/v1/audio-analysis/1vcq...,174707,4
130,0.347,0.381,8,-11.169,1,0.0302,0.84900,0.000031,0.2280,0.296,89.159,audio_features,3yuJ5HOIXe4ag9fQ9d1dmD,spotify:track:3yuJ5HOIXe4ag9fQ9d1dmD,https://api.spotify.com/v1/tracks/3yuJ5HOIXe4a...,https://api.spotify.com/v1/audio-analysis/3yuJ...,185813,4
131,0.671,0.725,2,-10.065,1,0.0650,0.29600,0.000220,0.0889,0.956,85.270,audio_features,1FRP8d6l2jm3DS5f78ZrhK,spotify:track:1FRP8d6l2jm3DS5f78ZrhK,https://api.spotify.com/v1/tracks/1FRP8d6l2jm3...,https://api.spotify.com/v1/audio-analysis/1FRP...,140400,4
132,0.657,0.825,0,-8.940,1,0.0980,0.21900,0.845000,0.3540,0.582,126.817,audio_features,7j96wehhMtN0fkVvFhD8Ix,spotify:track:7j96wehhMtN0fkVvFhD8Ix,https://api.spotify.com/v1/tracks/7j96wehhMtN0...,https://api.spotify.com/v1/audio-analysis/7j96...,137667,4


In [45]:
#export to csv
best_of_60s_af_df.to_csv("../data/the_best_of_60s_af.csv")

### 1000 of the best rock songs of all time

In [36]:
best_rock_all_time_playlist_id = "0Y8LL8xygULBsQnNWQgHiL"
best_rock_all_time_username = "bimsalasimm"
best_rock_all_time_af_df = playlist_audio_features(best_rock_all_time_username,best_rock_all_time_playlist_id)
best_rock_all_time_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.671,0.712,2,-7.815,1,0.0476,0.429000,0.000000,0.1030,0.462,113.805,audio_features,2fuCquhmrzHpu5xcA1ci9x,spotify:track:2fuCquhmrzHpu5xcA1ci9x,https://api.spotify.com/v1/tracks/2fuCquhmrzHp...,https://api.spotify.com/v1/audio-analysis/2fuC...,248440,4
1,0.511,0.632,6,-9.645,1,0.0439,0.012000,0.099600,0.3060,0.483,97.612,audio_features,4HCWqhZGWPWwryXmQ6D4UU,spotify:track:4HCWqhZGWPWwryXmQ6D4UU,https://api.spotify.com/v1/tracks/4HCWqhZGWPWw...,https://api.spotify.com/v1/audio-analysis/4HCW...,240093,4
2,0.204,0.837,5,-7.618,1,0.0676,0.281000,0.004050,0.2910,0.392,171.514,audio_features,0VafPclbphrwh36WraHWws,spotify:track:0VafPclbphrwh36WraHWws,https://api.spotify.com/v1/tracks/0VafPclbphrw...,https://api.spotify.com/v1/audio-analysis/0Vaf...,590000,4
3,0.500,0.851,2,-6.027,1,0.0380,0.012800,0.006980,0.1360,0.358,125.599,audio_features,2TlXPShOrPn4K51fP0Fykd,spotify:track:2TlXPShOrPn4K51fP0Fykd,https://api.spotify.com/v1/tracks/2TlXPShOrPn4...,https://api.spotify.com/v1/audio-analysis/2TlX...,275040,4
4,0.685,0.633,9,-7.259,1,0.0377,0.013600,0.000106,0.0935,0.731,131.397,audio_features,1jM23jASF0JrhgqYhqCZfl,spotify:track:1jM23jASF0JrhgqYhqCZfl,https://api.spotify.com/v1/tracks/1jM23jASF0Jr...,https://api.spotify.com/v1/audio-analysis/1jM2...,317240,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,0.490,0.854,7,-7.489,1,0.0407,0.000258,0.000735,0.2990,0.443,132.032,audio_features,2rbcCco3PrSyMinVJdh61J,spotify:track:2rbcCco3PrSyMinVJdh61J,https://api.spotify.com/v1/tracks/2rbcCco3PrSy...,https://api.spotify.com/v1/audio-analysis/2rbc...,307960,4
1407,0.625,0.609,7,-8.318,0,0.0264,0.015100,0.063500,0.0909,0.888,126.584,audio_features,5gDrMfCS7UxMZF5PHslroV,spotify:track:5gDrMfCS7UxMZF5PHslroV,https://api.spotify.com/v1/tracks/5gDrMfCS7UxM...,https://api.spotify.com/v1/audio-analysis/5gDr...,213533,4
1408,0.395,0.843,8,-4.476,1,0.0374,0.004830,0.000000,0.0404,0.481,112.423,audio_features,0nnwn7LWHCAu09jfuH1xTA,spotify:track:0nnwn7LWHCAu09jfuH1xTA,https://api.spotify.com/v1/tracks/0nnwn7LWHCAu...,https://api.spotify.com/v1/audio-analysis/0nnw...,212067,4
1409,0.605,0.742,4,-4.363,0,0.0327,0.002860,0.001050,0.2560,0.470,105.040,audio_features,4tXr9v3K7nWW1aebWqR1B7,spotify:track:4tXr9v3K7nWW1aebWqR1B7,https://api.spotify.com/v1/tracks/4tXr9v3K7nWW...,https://api.spotify.com/v1/audio-analysis/4tXr...,271160,4


In [37]:
#export to csv
best_rock_all_time_af_df.to_csv("../data/the_best_rock_all_time_af.csv")

### Guilty pleasures

In [38]:
guilty_pleasures_playlist_id = "3GnAT8Z3WNcon2uh6eVIiA"
guilty_pleasures_username = "janvelthuis1960"
guilty_pleasures_af_df = playlist_audio_features(guilty_pleasures_username,guilty_pleasures_playlist_id)
guilty_pleasures_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.548,0.258,11,-8.874,1,0.0293,0.47500,0.000029,0.4590,0.644,126.921,audio_features,6PHf8qq1n8mWplJ9nEZ9ED,spotify:track:6PHf8qq1n8mWplJ9nEZ9ED,https://api.spotify.com/v1/tracks/6PHf8qq1n8mW...,https://api.spotify.com/v1/audio-analysis/6PHf...,200293,4
1,0.768,0.864,11,-6.145,1,0.0324,0.10100,0.000000,0.1620,0.891,110.008,audio_features,1Je1IMUlBXcx1Fz0WE7oPT,spotify:track:1Je1IMUlBXcx1Fz0WE7oPT,https://api.spotify.com/v1/tracks/1Je1IMUlBXcx...,https://api.spotify.com/v1/audio-analysis/1Je1...,173027,4
2,0.721,0.939,8,-11.823,1,0.0376,0.11500,0.000038,0.1080,0.914,113.309,audio_features,4PTG3Z6ehGkBFwjybzWkR8,spotify:track:4PTG3Z6ehGkBFwjybzWkR8,https://api.spotify.com/v1/tracks/4PTG3Z6ehGkB...,https://api.spotify.com/v1/audio-analysis/4PTG...,213573,4
3,0.543,0.870,9,-6.514,1,0.0428,0.35800,0.000939,0.7920,0.754,100.804,audio_features,0GjEhVFGZW8afUYGChu3Rr,spotify:track:0GjEhVFGZW8afUYGChu3Rr,https://api.spotify.com/v1/tracks/0GjEhVFGZW8a...,https://api.spotify.com/v1/audio-analysis/0GjE...,230400,4
4,0.798,0.740,11,-4.865,1,0.0458,0.04190,0.000002,0.1570,0.834,128.054,audio_features,1cSXzDZt8vzuUp2XREQEJN,spotify:track:1cSXzDZt8vzuUp2XREQEJN,https://api.spotify.com/v1/tracks/1cSXzDZt8vzu...,https://api.spotify.com/v1/audio-analysis/1cSX...,219720,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,0.841,0.655,8,-8.150,1,0.2390,0.20800,0.000003,0.2910,0.865,108.442,audio_features,6Bne501HS96nEeNq8Gn87R,spotify:track:6Bne501HS96nEeNq8Gn87R,https://api.spotify.com/v1/tracks/6Bne501HS96n...,https://api.spotify.com/v1/audio-analysis/6Bne...,170507,4
760,0.553,0.702,2,-4.829,1,0.1080,0.00532,0.000005,0.0446,0.453,201.868,audio_features,3px156RqKPoqmxq44b3pSD,spotify:track:3px156RqKPoqmxq44b3pSD,https://api.spotify.com/v1/tracks/3px156RqKPoq...,https://api.spotify.com/v1/audio-analysis/3px1...,240200,4
761,0.667,0.753,5,-9.196,1,0.0543,0.01050,0.000000,0.3370,0.785,123.463,audio_features,2sHmscm6NClVLvPtqk1A73,spotify:track:2sHmscm6NClVLvPtqk1A73,https://api.spotify.com/v1/tracks/2sHmscm6NClV...,https://api.spotify.com/v1/audio-analysis/2sHm...,264653,4
762,0.849,0.875,0,-6.236,1,0.0300,0.09470,0.021300,0.3640,0.898,130.013,audio_features,4IiKITgHYaHrF1zUlcTyVH,spotify:track:4IiKITgHYaHrF1zUlcTyVH,https://api.spotify.com/v1/tracks/4IiKITgHYaHr...,https://api.spotify.com/v1/audio-analysis/4IiK...,230933,4


In [39]:
#export to csv
guilty_pleasures_af_df.to_csv("../data/guilty_pleasures_af.csv")

### Top 5000 pop songs

In [40]:
top_5000_pop_playlist_id = "4RVf1hHtwvMEED3yuCNi8q"
top_5000_pop_username = "1165808128"
top_5000_pop_af_df = playlist_audio_features(top_5000_pop_username,top_5000_pop_playlist_id)
top_5000_pop_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.429,0.554,9,-6.128,0,0.0291,0.6700,0.000152,0.1050,0.407,136.302,audio_features,1GcVa4jFySlun4jLSuMhiq,spotify:track:1GcVa4jFySlun4jLSuMhiq,https://api.spotify.com/v1/tracks/1GcVa4jFySlu...,https://api.spotify.com/v1/audio-analysis/1GcV...,272000,4
1,0.920,0.654,11,-3.051,0,0.0401,0.0236,0.015300,0.0360,0.847,117.046,audio_features,5ChkMS8OtdzJeqyybCc9R5,spotify:track:5ChkMS8OtdzJeqyybCc9R5,https://api.spotify.com/v1/tracks/5ChkMS8OtdzJ...,https://api.spotify.com/v1/audio-analysis/5Chk...,293827,4
2,0.386,0.607,10,-7.700,1,0.0261,0.0112,0.000014,0.0880,0.532,147.207,audio_features,0aym2LBJBk9DAYuHHutrIl,spotify:track:0aym2LBJBk9DAYuHHutrIl,https://api.spotify.com/v1/tracks/0aym2LBJBk9D...,https://api.spotify.com/v1/audio-analysis/0aym...,425653,4
3,0.367,0.452,10,-10.422,1,0.0307,0.0353,0.002280,0.6890,0.189,113.066,audio_features,3lUjT0JcV1PcuugwMl2MQj,spotify:track:3lUjT0JcV1PcuugwMl2MQj,https://api.spotify.com/v1/tracks/3lUjT0JcV1Pc...,https://api.spotify.com/v1/audio-analysis/3lUj...,520787,4
4,0.711,0.362,3,-10.851,0,0.0381,0.2720,0.000000,0.1270,0.616,117.287,audio_features,1tqT6DhmsrtQgyCKUwotiw,spotify:track:1tqT6DhmsrtQgyCKUwotiw,https://api.spotify.com/v1/tracks/1tqT6DhmsrtQ...,https://api.spotify.com/v1/audio-analysis/1tqT...,194293,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,0.690,0.843,9,-9.394,1,0.0352,0.5760,0.012400,0.1160,0.739,110.808,audio_features,5RqCmbI9RfoJu1oSXk3UTi,spotify:track:5RqCmbI9RfoJu1oSXk3UTi,https://api.spotify.com/v1/tracks/5RqCmbI9RfoJ...,https://api.spotify.com/v1/audio-analysis/5RqC...,359960,4
3646,0.653,0.658,1,-10.380,1,0.0335,0.2250,0.001080,0.0599,0.575,122.074,audio_features,0WujVnyLeCEOoPpdV2wOj6,spotify:track:0WujVnyLeCEOoPpdV2wOj6,https://api.spotify.com/v1/tracks/0WujVnyLeCEO...,https://api.spotify.com/v1/audio-analysis/0Wuj...,256267,4
3647,0.745,0.560,8,-9.009,0,0.0269,0.1720,0.000124,0.1530,0.925,105.634,audio_features,3OiMyoS1QCu8ozdDEj1WWD,spotify:track:3OiMyoS1QCu8ozdDEj1WWD,https://api.spotify.com/v1/tracks/3OiMyoS1QCu8...,https://api.spotify.com/v1/audio-analysis/3OiM...,277707,4
3648,0.686,0.755,1,-8.304,0,0.0488,0.1490,0.063000,0.1110,0.686,119.895,audio_features,5wdlG60d0WHoo8P3QzrlbG,spotify:track:5wdlG60d0WHoo8P3QzrlbG,https://api.spotify.com/v1/tracks/5wdlG60d0WHo...,https://api.spotify.com/v1/audio-analysis/5wdl...,470507,4


In [41]:
#export to csv
top_5000_pop_af_df.to_csv("../data/top_5000_pop_af.csv")

### 1000 greatest songs of all time

In [45]:
greatest_songs_all_time_playlist_id = "2LOxEzC4KmoWJ9NhW0kz5M"
greatest_songs_all_time_username = "maxaustralia"
greatest_songs_all_time_af_df = playlist_audio_features(greatest_songs_all_time_username,greatest_songs_all_time_playlist_id)
greatest_songs_all_time_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.414,0.404,0,-9.928,0,0.0499,0.271000,0.000000,0.3000,0.224,71.105,audio_features,4u7EnebtmKWzUH433cf5Qv,spotify:track:4u7EnebtmKWzUH433cf5Qv,https://api.spotify.com/v1/tracks/4u7EnebtmKWz...,https://api.spotify.com/v1/audio-analysis/4u7E...,354320,4
1,0.446,0.952,6,-5.321,1,0.0523,0.090100,0.085700,0.1120,0.624,125.303,audio_features,7snQQk1zcKl8gZ92AnueZW,spotify:track:7snQQk1zcKl8gZ92AnueZW,https://api.spotify.com/v1/tracks/7snQQk1zcKl8...,https://api.spotify.com/v1/audio-analysis/7snQ...,356067,4
2,0.502,0.912,1,-4.556,1,0.0564,0.000025,0.000173,0.1060,0.720,116.761,audio_features,4CeeEOM32jQcH3eN9Q2dGj,spotify:track:4CeeEOM32jQcH3eN9Q2dGj,https://api.spotify.com/v1/tracks/4CeeEOM32jQc...,https://api.spotify.com/v1/audio-analysis/4Cee...,301920,4
3,0.547,0.257,0,-12.358,1,0.0252,0.907000,0.183000,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867,4
4,0.369,0.762,10,-8.993,1,0.0464,0.220000,0.000028,0.5790,0.644,84.957,audio_features,5NwkWwfRJaT55hEPtCmJHx,spotify:track:5NwkWwfRJaT55hEPtCmJHx,https://api.spotify.com/v1/tracks/5NwkWwfRJaT5...,https://api.spotify.com/v1/audio-analysis/5Nwk...,302107,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,0.531,0.948,9,-5.438,1,0.0568,0.006000,0.011700,0.6940,0.498,136.891,audio_features,3UrDYCWfY6fTsiA63NNL27,spotify:track:3UrDYCWfY6fTsiA63NNL27,https://api.spotify.com/v1/tracks/3UrDYCWfY6fT...,https://api.spotify.com/v1/audio-analysis/3UrD...,298093,4
988,0.558,0.768,2,-9.997,1,0.0343,0.121000,0.062600,0.1240,0.429,81.909,audio_features,44aTAUBF0g6sMkMNE8I5kd,spotify:track:44aTAUBF0g6sMkMNE8I5kd,https://api.spotify.com/v1/tracks/44aTAUBF0g6s...,https://api.spotify.com/v1/audio-analysis/44aT...,260573,4
989,0.535,0.505,2,-8.926,0,0.2450,0.039300,0.000000,0.0923,0.495,99.935,audio_features,0uEp9E98JB5awlA084uaIg,spotify:track:0uEp9E98JB5awlA084uaIg,https://api.spotify.com/v1/tracks/0uEp9E98JB5a...,https://api.spotify.com/v1/audio-analysis/0uEp...,320267,4
990,0.445,0.706,8,-9.803,1,0.0461,0.234000,0.000252,0.2050,0.768,80.823,audio_features,43DeSV93pJPT4lCZaWZ6b1,spotify:track:43DeSV93pJPT4lCZaWZ6b1,https://api.spotify.com/v1/tracks/43DeSV93pJPT...,https://api.spotify.com/v1/audio-analysis/43De...,266720,4


In [46]:
#export to csv
greatest_songs_all_time_af_df.to_csv("../data/greatest_songs_all_time_af.csv")

### Longest playlist ever

In [59]:
longest_playlist_ever_playlist_id = "6yPiKpy7evrwvZodByKvM9"
longest_playlist_ever_username = "12160726861"
longest_playlist_ever_af_df = playlist_audio_features(longest_playlist_ever_username,longest_playlist_ever_playlist_id)
longest_playlist_ever_af_df

AttributeError: 'NoneType' object has no attribute 'keys'

Note: my function 'playlist audio feature' returned an error for this playlist. below is the investigative work on why this happened

#### Investigation

In [8]:
longest_playlist_ever_test_playlist_id = "6yPiKpy7evrwvZodByKvM9"
longest_playlist_ever_test_username = "12160726861"

tracks = get_playlist_tracks(longest_playlist_ever_test_username,longest_playlist_ever_test_playlist_id)


In [14]:
#for all
track_ids = []
for i in tracks:
    track_ids.append(i['track']['id'])


Note: identified 'None' values in list of track ids. I believe this is why my function was returning an error. I believe song on spotify being "unavailable" results in a track_id of 'none'

In [18]:
#removing null values
filtered_track_ids = [item for item in track_ids if item is not None]


In [19]:
#Chopping into chunks
chunk_size = 100
filtered_track_ids_chopped = [filtered_track_ids[i:i+chunk_size] for i in range(0,len(filtered_track_ids), 100)]

In [20]:
#getting audio features
af_list_2 = []
for i in filtered_track_ids_chopped:
    af_list_2.extend(sp.audio_features(i))

In [21]:
af_list_2_filtered = [item for item in af_list_2 if item is not None]

In [22]:
#creating data frame
longest_playlist_ever_af_df_2 = pd.DataFrame(af_list_2_filtered)


In [23]:
longest_playlist_ever_af_df_2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.406,0.445,7,-13.355,0,0.0745,0.09300,0.072100,0.1130,0.123,134.972,audio_features,3lMSWyB8H0A0rSRhUDSwoz,spotify:track:3lMSWyB8H0A0rSRhUDSwoz,https://api.spotify.com/v1/tracks/3lMSWyB8H0A0...,https://api.spotify.com/v1/audio-analysis/3lMS...,152560,4
1,0.419,0.565,5,-11.565,1,0.0347,0.13700,0.337000,0.1280,0.648,151.277,audio_features,3UnyplmZaq547hwsfOR5yy,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707,4
2,0.532,0.303,9,-13.076,1,0.0311,0.58700,0.065700,0.1110,0.225,124.157,audio_features,3HPLfZhQUds6N6tZ4WiApn,spotify:track:3HPLfZhQUds6N6tZ4WiApn,https://api.spotify.com/v1/tracks/3HPLfZhQUds6...,https://api.spotify.com/v1/audio-analysis/3HPL...,244285,4
3,0.717,0.800,5,-5.794,0,0.0497,0.37100,0.158000,0.3270,0.954,129.642,audio_features,5iFhkFf9JLA3XiROdYsg1i,spotify:track:5iFhkFf9JLA3XiROdYsg1i,https://api.spotify.com/v1/tracks/5iFhkFf9JLA3...,https://api.spotify.com/v1/audio-analysis/5iFh...,250000,4
4,0.571,0.724,0,-5.967,1,0.0260,0.02310,0.003110,0.0919,0.385,116.755,audio_features,1VZedwJj1gyi88WFRhfThb,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9912,0.651,0.801,0,-7.340,1,0.0514,0.12300,0.000000,0.0825,0.776,133.769,audio_features,5jzma6gCzYtKB1DbEwFZKH,spotify:track:5jzma6gCzYtKB1DbEwFZKH,https://api.spotify.com/v1/tracks/5jzma6gCzYtK...,https://api.spotify.com/v1/audio-analysis/5jzm...,200480,4
9913,0.812,0.644,0,-10.761,1,0.0493,0.08770,0.785000,0.0563,0.989,140.075,audio_features,7Bz8yww6UMbTgTVLG6zbI4,spotify:track:7Bz8yww6UMbTgTVLG6zbI4,https://api.spotify.com/v1/tracks/7Bz8yww6UMbT...,https://api.spotify.com/v1/audio-analysis/7Bz8...,190827,4
9914,0.687,0.606,7,-5.920,1,0.0262,0.17800,0.000326,0.0830,0.748,80.569,audio_features,0jpk88zk40MjQ63ljrq7V2,spotify:track:0jpk88zk40MjQ63ljrq7V2,https://api.spotify.com/v1/tracks/0jpk88zk40Mj...,https://api.spotify.com/v1/audio-analysis/0jpk...,206307,4
9915,0.711,0.718,6,-5.739,1,0.0380,0.00824,0.002080,0.0732,0.688,117.071,audio_features,0sEm1ld0V8YTCPcjPVfIsc,spotify:track:0sEm1ld0V8YTCPcjPVfIsc,https://api.spotify.com/v1/tracks/0sEm1ld0V8YT...,https://api.spotify.com/v1/audio-analysis/0sEm...,200293,4


In [24]:
#export to csv
longest_playlist_ever_af_df_2.to_csv("../data/longest_playlist_ever_af.csv")

### Worlds largest playlist

In [52]:
worlds_largest_playlist_id = "4vBaGi0msaCBxhAq9lWXk3"
worlds_largest_username = "larrypeay"
worlds_largest_af_df = playlist_audio_features(worlds_largest_username,worlds_largest_playlist_id)
worlds_largest_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.485,0.721,5,-7.840,0,0.0348,0.4790,0.000000,0.6700,0.880,136.892,audio_features,6rBgZvSsyJrk5ZXtKc8mcf,spotify:track:6rBgZvSsyJrk5ZXtKc8mcf,https://api.spotify.com/v1/tracks/6rBgZvSsyJrk...,https://api.spotify.com/v1/audio-analysis/6rBg...,142573,4
1,0.402,0.643,5,-8.749,1,0.0317,0.2400,0.000039,0.1180,0.525,150.086,audio_features,5OuaAMBmGjjJMK7yXpaFAK,spotify:track:5OuaAMBmGjjJMK7yXpaFAK,https://api.spotify.com/v1/tracks/5OuaAMBmGjjJ...,https://api.spotify.com/v1/audio-analysis/5Oua...,199707,4
2,0.489,0.561,4,-7.933,1,0.0274,0.2000,0.000046,0.1790,0.238,83.457,audio_features,1KHdq8NK9QxnGjdXb55NiG,spotify:track:1KHdq8NK9QxnGjdXb55NiG,https://api.spotify.com/v1/tracks/1KHdq8NK9Qxn...,https://api.spotify.com/v1/audio-analysis/1KHd...,244987,3
3,0.568,0.634,4,-12.336,1,0.0282,0.4940,0.002810,0.2520,0.866,142.686,audio_features,40h65HAR8COEoqkMwUUQHu,spotify:track:40h65HAR8COEoqkMwUUQHu,https://api.spotify.com/v1/tracks/40h65HAR8COE...,https://api.spotify.com/v1/audio-analysis/40h6...,257963,4
4,0.714,0.886,11,-8.629,0,0.0554,0.1160,0.184000,0.1800,0.954,156.351,audio_features,3oAWTk92mZBxKBOKf8mR5v,spotify:track:3oAWTk92mZBxKBOKf8mR5v,https://api.spotify.com/v1/tracks/3oAWTk92mZBx...,https://api.spotify.com/v1/audio-analysis/3oAW...,119360,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,0.660,0.639,9,-12.749,1,0.0369,0.0966,0.001040,0.0442,0.843,84.124,audio_features,3PJ9yQv8IUmLx24KFvaVEI,spotify:track:3PJ9yQv8IUmLx24KFvaVEI,https://api.spotify.com/v1/tracks/3PJ9yQv8IUmL...,https://api.spotify.com/v1/audio-analysis/3PJ9...,194667,4
5596,0.511,0.308,7,-15.021,1,0.0247,0.4080,0.000024,0.1500,0.380,83.545,audio_features,2Xa2C6ftMdcg3j7YSot8dY,spotify:track:2Xa2C6ftMdcg3j7YSot8dY,https://api.spotify.com/v1/tracks/2Xa2C6ftMdcg...,https://api.spotify.com/v1/audio-analysis/2Xa2...,235560,4
5597,0.612,0.876,0,-7.836,1,0.0340,0.0949,0.000045,0.0764,0.971,135.106,audio_features,5hrNXBpS7BDrZtIsqNpReY,spotify:track:5hrNXBpS7BDrZtIsqNpReY,https://api.spotify.com/v1/tracks/5hrNXBpS7BDr...,https://api.spotify.com/v1/audio-analysis/5hrN...,249827,4
5598,0.620,0.427,0,-13.819,1,0.0294,0.7330,0.000142,0.0929,0.931,87.777,audio_features,2ons8YrupZvbMgJoL3zy77,spotify:track:2ons8YrupZvbMgJoL3zy77,https://api.spotify.com/v1/tracks/2ons8YrupZvb...,https://api.spotify.com/v1/audio-analysis/2ons...,192200,4


In [53]:
#export to csv
worlds_largest_af_df.to_csv("../data/worlds_largest_af.csv")

### R&B Throwbacks 90s & 00s

In [25]:
r_and_b_throwbacks_playlist_id = "7s17jTD83GgeKajAMogIen"
r_and_b_throwbacks_username = "12145397198"
r_and_b_throwbacks_af_df = playlist_audio_features(r_and_b_throwbacks_username,r_and_b_throwbacks_playlist_id)
r_and_b_throwbacks_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.727,0.552,2,-8.074,0,0.1400,0.2270,0.000164,0.1980,0.607,168.189,audio_features,0ARK753YaiJbpLUk7z5yIM,spotify:track:0ARK753YaiJbpLUk7z5yIM,https://api.spotify.com/v1/tracks/0ARK753YaiJb...,https://api.spotify.com/v1/audio-analysis/0ARK...,289160,4
1,0.622,0.593,7,-8.352,0,0.0916,0.1630,0.000000,0.0979,0.607,94.364,audio_features,5WphWTUIfRe7x8NZss79cY,spotify:track:5WphWTUIfRe7x8NZss79cY,https://api.spotify.com/v1/tracks/5WphWTUIfRe7...,https://api.spotify.com/v1/audio-analysis/5Wph...,249093,4
2,0.684,0.455,5,-8.216,1,0.0307,0.2150,0.000001,0.2630,0.487,97.956,audio_features,174rZBKJAqD10VBnOjlQQ3,spotify:track:174rZBKJAqD10VBnOjlQQ3,https://api.spotify.com/v1/tracks/174rZBKJAqD1...,https://api.spotify.com/v1/audio-analysis/174r...,216893,4
3,0.536,0.612,4,-5.847,1,0.2720,0.1190,0.000000,0.2090,0.570,86.768,audio_features,7DFnq8FYhHMCylykf6ZCxA,spotify:track:7DFnq8FYhHMCylykf6ZCxA,https://api.spotify.com/v1/tracks/7DFnq8FYhHMC...,https://api.spotify.com/v1/audio-analysis/7DFn...,229040,4
4,0.412,0.876,5,-6.008,0,0.4700,0.1020,0.000000,0.2970,0.703,130.159,audio_features,5Y77SQxEr1eiofPeUTPHxM,spotify:track:5Y77SQxEr1eiofPeUTPHxM,https://api.spotify.com/v1/tracks/5Y77SQxEr1ei...,https://api.spotify.com/v1/audio-analysis/5Y77...,273133,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,0.468,0.723,1,-5.344,1,0.0782,0.1900,0.000000,0.3660,0.449,178.860,audio_features,1PzVLh1uoq5pAUnqNQElQq,spotify:track:1PzVLh1uoq5pAUnqNQElQq,https://api.spotify.com/v1/tracks/1PzVLh1uoq5p...,https://api.spotify.com/v1/audio-analysis/1PzV...,181027,4
292,0.730,0.602,6,-3.782,0,0.2060,0.3620,0.000004,0.1690,0.927,97.954,audio_features,69XUpOpjzDKcfdxqZebGiI,spotify:track:69XUpOpjzDKcfdxqZebGiI,https://api.spotify.com/v1/tracks/69XUpOpjzDKc...,https://api.spotify.com/v1/audio-analysis/69XU...,221133,4
293,0.794,0.662,6,-4.913,0,0.2490,0.0789,0.000000,0.0937,0.683,93.053,audio_features,6Wnlq1Pu3I0BajLvxn05Vr,spotify:track:6Wnlq1Pu3I0BajLvxn05Vr,https://api.spotify.com/v1/tracks/6Wnlq1Pu3I0B...,https://api.spotify.com/v1/audio-analysis/6Wnl...,284640,4
294,0.764,0.821,4,-6.095,0,0.1940,0.0922,0.000000,0.1590,0.962,162.495,audio_features,75ABQumvUWSaUsEGjZFu3G,spotify:track:75ABQumvUWSaUsEGjZFu3G,https://api.spotify.com/v1/tracks/75ABQumvUWSa...,https://api.spotify.com/v1/audio-analysis/75AB...,282173,4


In [26]:
#export to csv
r_and_b_throwbacks_af_df.to_csv("../data/r_and_b_throwbacks_af.csv")

### Billboard_hot_100

In [15]:
billboard_hot_100_playlist_id = "6UeSakyzhiEt4NB3UAd6NQ"
billboard_hot_100_username = "billboard.com"
billboard_hot_100_af_df = playlist_audio_features(billboard_hot_100_username,billboard_hot_100_playlist_id)
billboard_hot_100_af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.745,0.663,1,-5.056,1,0.4230,0.0780,0.000000,0.2130,0.686,167.137,audio_features,79AHPJd8upvNxyK2qQcY8J,spotify:track:79AHPJd8upvNxyK2qQcY8J,https://api.spotify.com/v1/tracks/79AHPJd8upvN...,https://api.spotify.com/v1/audio-analysis/79AH...,192819,4
1,0.943,0.558,2,-4.911,1,0.0568,0.0026,0.000002,0.0937,0.606,104.983,audio_features,4xhsWYTOGcal8zt0J161CU,spotify:track:4xhsWYTOGcal8zt0J161CU,https://api.spotify.com/v1/tracks/4xhsWYTOGcal...,https://api.spotify.com/v1/audio-analysis/4xhs...,138411,4
2,0.552,0.702,9,-5.707,1,0.1570,0.1170,0.000021,0.1050,0.564,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427,4
3,0.561,0.604,9,-4.409,1,0.0337,0.1990,0.000019,0.1040,0.242,159.920,audio_features,6usohdchdzW9oML7VC4Uhk,spotify:track:6usohdchdzW9oML7VC4Uhk,https://api.spotify.com/v1/tracks/6usohdchdzW9...,https://api.spotify.com/v1/audio-analysis/6uso...,210689,3
4,0.750,0.733,6,-3.180,0,0.0319,0.2560,0.000000,0.1140,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.431,0.628,10,-6.259,0,0.0384,0.4030,0.000008,0.2720,0.277,118.016,audio_features,6RWCvgNCEqXuVZYnDTPDEh,spotify:track:6RWCvgNCEqXuVZYnDTPDEh,https://api.spotify.com/v1/tracks/6RWCvgNCEqXu...,https://api.spotify.com/v1/audio-analysis/6RWC...,250880,4
96,0.486,0.640,3,-8.027,1,0.0551,0.1360,0.000076,0.1050,0.289,122.017,audio_features,4oEf84vBYVftf6KmZexhVo,spotify:track:4oEf84vBYVftf6KmZexhVo,https://api.spotify.com/v1/tracks/4oEf84vBYVft...,https://api.spotify.com/v1/audio-analysis/4oEf...,267256,4
97,0.606,0.757,1,-6.206,1,0.0302,0.0157,0.000205,0.3430,0.588,103.051,audio_features,0lbHiQaHPlp5gGtsUPtj6l,spotify:track:0lbHiQaHPlp5gGtsUPtj6l,https://api.spotify.com/v1/tracks/0lbHiQaHPlp5...,https://api.spotify.com/v1/audio-analysis/0lbH...,162345,4
98,0.455,0.621,0,-7.694,0,0.0517,0.6510,0.000003,0.1230,0.280,119.043,audio_features,4kjI1gwQZRKNDkw1nI475M,spotify:track:4kjI1gwQZRKNDkw1nI475M,https://api.spotify.com/v1/tracks/4kjI1gwQZRKN...,https://api.spotify.com/v1/audio-analysis/4kjI...,251250,4


In [16]:
#export to csv
billboard_hot_100_af_df.to_csv("../data/billboard_hot_100_af.csv")

## Final dataframe

In [8]:
#Import latest dataframe
full_af_from_csv = pd.read_csv('../data/full_af.csv')

In [9]:
full_af = pd.concat([full_af_from_csv,greatest_songs_all_time_af_df,worlds_largest_af_df,r_and_b_throwbacks_af_df], ignore_index=True)
full_af

NameError: name 'greatest_songs_all_time_af_df' is not defined

In [55]:
#export to csv
full_af.to_csv("../data/full_af.csv")

In [56]:
full_af = full_af.drop_duplicates()
full_af

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.382,0.451,7,-6.606,0,0.0293,0.5570,0.000034,0.2650,0.506,150.803,audio_features,6yzJiHzsscf0n88xb9aNT4,spotify:track:6yzJiHzsscf0n88xb9aNT4,https://api.spotify.com/v1/tracks/6yzJiHzsscf0...,https://api.spotify.com/v1/audio-analysis/6yzJ...,267540,4
1,0.742,0.375,7,-13.829,1,0.0482,0.3970,0.901000,0.0996,0.105,102.023,audio_features,5zdXRoKLZ54VBdbdn7yhz4,spotify:track:5zdXRoKLZ54VBdbdn7yhz4,https://api.spotify.com/v1/tracks/5zdXRoKLZ54V...,https://api.spotify.com/v1/audio-analysis/5zdX...,342163,4
2,0.585,0.552,0,-10.613,1,0.0353,0.1940,0.017100,0.0735,0.496,100.089,audio_features,5qvCKAyjgHnpR5c63OedCY,spotify:track:5qvCKAyjgHnpR5c63OedCY,https://api.spotify.com/v1/tracks/5qvCKAyjgHnp...,https://api.spotify.com/v1/audio-analysis/5qvC...,267813,4
3,0.405,0.535,6,-7.504,0,0.0531,0.0531,0.106000,0.0696,0.637,97.698,audio_features,2vByA9JgfMU2bPjUqmxCdv,spotify:track:2vByA9JgfMU2bPjUqmxCdv,https://api.spotify.com/v1/tracks/2vByA9JgfMU2...,https://api.spotify.com/v1/audio-analysis/2vBy...,210800,4
4,0.586,0.526,0,-6.928,1,0.0564,0.5550,0.000000,0.2340,0.456,107.153,audio_features,0pPQwIjwqduvLfZORYYEaI,spotify:track:0pPQwIjwqduvLfZORYYEaI,https://api.spotify.com/v1/tracks/0pPQwIjwqduv...,https://api.spotify.com/v1/audio-analysis/0pPQ...,161267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13943,0.660,0.639,9,-12.749,1,0.0369,0.0966,0.001040,0.0442,0.843,84.124,audio_features,3PJ9yQv8IUmLx24KFvaVEI,spotify:track:3PJ9yQv8IUmLx24KFvaVEI,https://api.spotify.com/v1/tracks/3PJ9yQv8IUmL...,https://api.spotify.com/v1/audio-analysis/3PJ9...,194667,4
13944,0.511,0.308,7,-15.021,1,0.0247,0.4080,0.000024,0.1500,0.380,83.545,audio_features,2Xa2C6ftMdcg3j7YSot8dY,spotify:track:2Xa2C6ftMdcg3j7YSot8dY,https://api.spotify.com/v1/tracks/2Xa2C6ftMdcg...,https://api.spotify.com/v1/audio-analysis/2Xa2...,235560,4
13945,0.612,0.876,0,-7.836,1,0.0340,0.0949,0.000045,0.0764,0.971,135.106,audio_features,5hrNXBpS7BDrZtIsqNpReY,spotify:track:5hrNXBpS7BDrZtIsqNpReY,https://api.spotify.com/v1/tracks/5hrNXBpS7BDr...,https://api.spotify.com/v1/audio-analysis/5hrN...,249827,4
13946,0.620,0.427,0,-13.819,1,0.0294,0.7330,0.000142,0.0929,0.931,87.777,audio_features,2ons8YrupZvbMgJoL3zy77,spotify:track:2ons8YrupZvbMgJoL3zy77,https://api.spotify.com/v1/tracks/2ons8YrupZvb...,https://api.spotify.com/v1/audio-analysis/2ons...,192200,4
